<img src="logo.png">

# Introducción a la Estadística

En este capítulo haremos una breve introducción a la Estadística descriptiva de **R**.

Comenzemos cargando el archivo csv ``vinos.csv``

In [ ]:
vinos  <- read.csv("vinos.csv")
names(vinos)

mex <- vinos[vinos$country == "Mexico",]
por <- vinos[vinos$country == "Portugal",]

## Visualización de distribuciones de datos

Dada una variable continua (es decir, un vector de tipo numérico), en **ggplot2** de **R** existe una función que nos permite graficar su *función de densidad de probabilidad*. Es decir, un histograma suavizado: ``geom_density``


In [ ]:
### Histograma suavizado de las calificaciones de los vinos en Portugal

ggplot() +
  geom_density(data=por,aes(x=as.numeric(points))) +
  geom_histogram(data=por,aes(x=as.numeric(points),y=stat(density)),
                 fill = "yellow",alpha=0.5,binwidth = 1)

De ese gráfico podemos deducir, por ejemplo, que la probabilidad de elegir un vino portugués al azar y que tenga calificación menor a 82.5 es baja, así como de tener vinos con calificación mayor a 95.

Observe además que hay 18 barras, cada una de ancho 1. Las nueve primeras quedan por atrás de 88.5. Como la cola izquierda es más ligera que la derecha, entonces en general hay mas vinos con calificación menor a 88.5 que mayor, lo cual podemos verificar.

In [ ]:
nrow(por[as.numeric(por$points)<88.5,])
nrow(por[as.numeric(por$points)>=88.5,])

## Media y mediana

La media de una característica es simplemente el valor promedio de sus datos. Se trata de una medida de tendencia central y de localización de la distribución, pues nos dice en qué valor los datos se están agrupando y además nos ayuda a saber aproximadamente en qué lugar del eje horizontal se encuentra esa aglomeración de datos.

Por su parte, la mediana de un característica es aquél número que nos divide a los datos en el 50%. En una distribución que tiene ambas colas (que son las más comunes), nos indica que si el lado derecho de la media es mas corto que el izquierdo, entonces tienes más valores superiores que inferiores (y por lo tanto, en los inferiores hay repeticiones).

Por lo tanto, entre más parecidas son la media y la mediana, la distribución es más simétrica. 

In [ ]:
## Caso México

median(as.numeric(mex$points))

nrow(mex[as.numeric(mex$points)<84,])
nrow(mex[as.numeric(mex$points)>=84,])

table(mex[as.numeric(mex$points)<84,"points"])
table(mex[as.numeric(mex$points)>=84,"points"])

Compara ambas distribuciones

In [ ]:
por1 <- data.frame(points = as.numeric(por$points),country = "Portugal")
mex1 <- data.frame(points = as.numeric(mex$points),country = "Mexico")

por_mex <- rbind(por1,mex1)

ggplot(data = por_mex) +
  geom_density(aes(points,color = country,fill=country),alpha=0.5)

Veamos en qué paises la media y la mediana son aproximadamente iguales, y por lo tanto sus curvas deben ser casi simétricas

In [ ]:
medias_medianas <- data.frame(medias = tapply(as.numeric(vinos$points),
                                              vinos$country,
                                              mean),
                              medianas = tapply(as.numeric(vinos$points),
                                                vinos$country,
                                                median)
)

medias_medianas$simetria <- ifelse(abs(medias_medianas$medias-medias_medianas$medianas)<0.5
  ,"Si","No") 

## Moda

Otra medida de tendencia central es la moda. Este valor es el que más se repite dentro de una característica. Cobra especial importancia cuando la caraterística es de tipo categórica.

Sin embargo, **R no tiene una función para calcular la moda**.

In [3]:
# Función para calcular la moda

moda <- function(vector){
    tabla <- as.data.frame(table(vector))
    tabla <- tabla[tabla$Freq == max(tabla$Freq),]
    names(tabla) <- c("Moda","Frecuencia")
    return(tabla)    
}

## Varianza y desviación estándar

La varianza y la desviación estándar de una característica nos dicen qué tanto se alejan (o dispersan) los datos de su promedio. Es por ello que se consideran medidas de dispersión.

Veamos un ejemplo. Compara los puntajes de los vinos en México, Chile y Portugal.

In [ ]:
# Tabla de los tres países
mis_paises <- vinos[vinos$country %in% c("Mexico","Portugal","Chile"),]

# Valor promedio de la calidad en cada país
medias = data.frame(puntaje_medio = tapply(mis_paises$points,
                mis_paises$country,
                function(x){mean(x,na.rm=TRUE)}))
medias$country <- row.names(medias)

# Graficas
ggplot() +
  geom_density(data = mis_paises,
               aes(x=points,fill=country),
               alpha = 0.5, 
               position = "identity") +
  geom_vline(data = medias,
             aes(xintercept = puntaje_medio,color=country)) +
  xlim(0,100)

Como dijimos, la desviación estándar mide qué tanto se alejan los valores, en promedio, de su propio promedio. Es decir, a menor desviación, mas "delgada" es la curva.

En **R**, la desviación estándar se calcula con la función ``sd()``.

In [ ]:
tapply(mis_paises$points,
       mis_paies$country,
       function(x){sd(x,na.rm=TRUE)})

Por lo tanto, a menor desviación, hay mas estabilidad de la característica de estudio. En este sentido, un país es productor de de vino con calidad estable (lo cual no significa que sea mejor) si su desviación es baja.

In [ ]:
## Buscaremos los países que tienen mas de 50 productos en la tabla
vinos$FAC <- 1

productos_paises <- data.frame(total = tapply(vinos$FAC,vinos$country,sum))
productos_paises$country <- row.names(productos_paises)

mayores <- productos_paises[productos_paises$total>50,]$country

mas_productores <- vinos[vinos$country %in% mayores,]

## Para cada uno de los países anteriores,
## calculamos la media y la desviación

calidad_estable <- data.frame(medias = tapply(mas_productores$points,
                          mas_productores$country, function(x){mean(x,na.rm = TRUE)}),
                              desviaciones = tapply(mas_productores$points,
                          mas_productores$country, function(x){sd(x,na.rm = TRUE)})
                              )
## Ordenamos el dataframe para ir de mas estables a menos estables y 
## de mayor calidad promedio a menor calidad promedio

calidad_estable$country  <- row.names(calidad_estable)
calidad_estable <- calidad_estable[order(calidad_estable$desviacion,
                                         calidad_estable$media,
                                         decreasing = c(FALSE,TRUE)),]
row.names(calidad_estable) <- 1:nrow(calidad_estable)

View(calidad_estable)
